# LLM API — Getting Started

This notebook shows how to call the **GPT-OSS** model hosted on `hub.qazcode.ai`.

The server uses the **OpenAI-compatible API**, so you can use the `openai` Python library or plain `requests`.

## 1. Install dependencies

In [ ]:
# !pip install openai

## 2. Configuration

In [6]:
API_KEY = "sk-kDGHTZAOX-jQcN8VXxQucg"  # replace with your key
HUB_URL = "https://hub.qazcode.ai/"

In [7]:
from openai import OpenAI

client = OpenAI(
    base_url=HUB_URL,
    api_key=API_KEY,  # replace with your key
)

MODEL = "oss-120b"

## 3. Basic Chat Completion

In [8]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "user", "content": "Hello! Who are you?"}
    ],
)

print(response.choices[0].message.content)

Hello! I’m ChatGPT — an AI language model created by OpenAI. I’m here to answer questions, brainstorm ideas, help with writing, explain concepts, or just chat about whatever’s on your mind. How can I assist you today?


## 4. System Prompt + User Message

In [9]:
response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {
            "role": "system",
            "content": "You are a medical diagnosis assistant. Given patient symptoms, suggest the most probable diagnosis with an ICD-10 code."
        },
        {
            "role": "user",
            "content": "Patient presents with fever, dry cough, and shortness of breath lasting 5 days."
        }
    ],
)

print(response.choices[0].message.content)

**Probable Diagnosis:**  
**COVID‑19, virus identified** (acute respiratory infection caused by SARS‑CoV‑2)

**ICD‑10‑CM Code:** **U07.1** – *COVID‑19, virus identified*

---

### Rationale
| Symptom | Typical in COVID‑19? | Comments |
|--------|----------------------|----------|
| Fever (5 days) | Very common (≈80‑90 % of cases) | Often low‑grade to high‑grade, may be intermittent |
| Dry cough | Classic presenting feature (≈60‑70 %) | Usually non‑productive in early disease |
| Shortness of breath (dyspnea) | Present in ≈30‑40 % of symptomatic patients, especially after several days of illness | Suggests lower‑tract involvement; may precede hypoxia |

The combination of **fever, dry cough, and new‑onset dyspnea** over a 5‑day period is highly consistent with an acute COVID‑19 infection, especially in the current epidemiologic context where SARS‑CoV‑2 remains a prevalent respiratory pathogen.

---

### Important Differential Diagnoses to Consider
| Condition | ICD‑10‑CM | Key distingu

## 5. Structured JSON Output

In [10]:
import json

SYSTEM_PROMPT = """You are a clinical decision support system.
Given patient symptoms, return a JSON object:
{
  "diagnoses": [
    {"rank": 1, "icd_code": "...", "name": "...", "explanation": "..."},
    ...
  ]
}
Return top 3 diagnoses ranked by likelihood. Respond with JSON only."""

symptoms = "Severe headache, nausea, vomiting, neck stiffness, sensitivity to light."

response = client.chat.completions.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": symptoms},
    ],
)

result = json.loads(response.choices[0].message.content)
for d in result["diagnoses"]:
    print(f"[{d['rank']}] {d['icd_code']} — {d['name']}")
    print(f"    {d['explanation']}\n")

[1] G00.9 — Bacterial meningitis, unspecified
    Classic triad of severe headache, nausea/vomiting, neck stiffness and photophobia strongly suggests acute meningitis. The rapid progression and systemic symptoms raise concern for bacterial etiology, which requires emergent antimicrobial therapy and lumbar puncture for confirmation.

[2] I60.9 — Nontraumatic subarachnoid hemorrhage, unspecified
    Sudden, severe (thunderclap) headache with associated neck stiffness and vomiting can mimic meningitis. A subarachnoid hemorrhage must be considered, especially if the headache onset was abrupt. Urgent non‑contrast CT head (or lumbar puncture if CT is negative) is indicated.

[3] G04.9 — Encephalitis, unspecified
    Encephalitis can present with headache, nausea, vomiting, meningismus and photophobia, often accompanied by altered mental status or seizures. Though less common than meningitis, it remains a serious differential, particularly if fever or focal neurologic signs develop.



In [13]:
import requests

response = requests.post(
    "https://{}/chat/completions".format(HUB_URL),
    headers={
        "Content-Type": "application/json",
        "Authorization": f"Bearer {API_KEY}",
    },
    json={
        "model": "oss-120b",
        "messages": [{"role": "user", "content": "Hello!"}],
    },
)

print(response.json()["choices"][0]["message"]["content"])

ConnectionError: HTTPSConnectionPool(host='https', port=443): Max retries exceeded with url: /hub.qazcode.ai//chat/completions (Caused by NameResolutionError("HTTPSConnection(host='https', port=443): Failed to resolve 'https' ([Errno 11001] getaddrinfo failed)"))

In [ ]:
import json
import networkx as nx
import matplotlib.pyplot as plt

# Путь к твоему файлу
INPUT_FILE = ".\\data\\processed\\protocol_features.jsonl"

# Собираем рёбра: (symptom, diagnosis)
edges = []

with open(INPUT_FILE, "r", encoding="utf-8") as f:
    for line in f:
        if not line.strip():
            continue
        data = json.loads(line)

        # --- ПОДПРАВЬ ЭТИ КЛЮЧИ ПОД СВОЙ ФОРМАТ ---
        diagnosis = data.get("diagnosis") or data.get("name") or data.get("disease")
        symptoms = data.get("symptoms", [])

        if not diagnosis or not symptoms:
            continue
        # ------------------------------------------

        for s in symptoms:
            edges.append((s, diagnosis))

# Строим граф
G = nx.Graph()
G.add_edges_from(edges)

# Разделяем узлы на симптомы и диагнозы
symptoms = {u for u, v in edges}
diagnoses = {v for u, v in edges}

# Раскраска и позиции
pos = nx.bipartite_layout(G, symptoms, align="vertical")

plt.figure(figsize=(16, 10))
nx.draw_networkx_nodes(
    G, pos,
    nodelist=symptoms,
    node_color="lightcoral",
    node_size=50,
    label="Symptoms"
)
nx.draw_networkx_nodes(
    G, pos,
    nodelist=diagnoses,
    node_color="lightblue",
    node_size=50,
    label="Diagnoses"
)
nx.draw_networkx_edges(G, pos, alpha=0.2, edge_color="gray")
# Можно убрать labels, если их слишком много
# nx.draw_networkx_labels(G, pos, font_size=6)

plt.title("Symptom–Diagnosis Network (Bipartite)")
plt.axis("off")
plt.tight_layout()
plt.show()
